In [24]:
# Import needed libraries
import pandas as pd
import numpy as np
from scipy.linalg import lstsq

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

In [25]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

# Disable scientific notation in display for ease of reading
pd.set_option('display.float_format', '{:.0f}'.format)


In [26]:
title_df = pd.read_csv("data/title.basics.tsv", sep='\t')
ratings_df = pd.read_csv("data/title.ratings.tsv", sep='\t')
# principals_df = pd.read_csv("data/title.principals.tsv", sep='\t')
# crew_df = pd.read_csv("data/title.crew.tsv", sep='\t')



In [27]:
print(title_df.columns)
title_cleaned = title_df.drop('endYear', axis=1) #dropppping end year, only relevant to tv shows
title_cleaned = title_cleaned.drop('titleType', axis=1) # drop title of type
title_cleaned = title_cleaned.drop('originalTitle', axis=1) # drop og title
title_cleaned = title_cleaned.drop('primaryTitle', axis=1) # drop primary used title

print(title_cleaned.columns)


Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')
Index(['tconst', 'isAdult', 'startYear', 'runtimeMinutes', 'genres'], dtype='object')


In [28]:
# print(title_cleaned['genres'].head(1))
# 
# title_cleaned['genres2'] = title_cleaned['genres'].apply(lambda x: x.split(",") if pd.notnull(x) and x != "\\N" else [])
title_cleaned['genres'] = title_cleaned['genres'].replace("\\N", "")
genre_dummies = title_cleaned['genres'].str.get_dummies(sep=",")
title_cleaned = pd.concat([title_cleaned, genre_dummies], axis=1)
title_cleaned = title_cleaned.drop("genres", axis=1)
merged = title_cleaned.merge(ratings_df, on="tconst", how="inner")
print(merged.columns)




# unique_genres = exploded['genres2'].unique()
# print(unique_genres)
# print(pd.unique(exploded_genres['genres']))
# print(len(pd.unique(exploded_genres)))

Index(['tconst', 'isAdult', 'startYear', 'runtimeMinutes', 'Action', 'Adult',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
       'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War',
       'Western', 'averageRating', 'numVotes'],
      dtype='object')


In [32]:
merged = merged.replace("\\N", np.nan)

X = merged.drop(columns=["averageRating", "tconst"])
X = X.select_dtypes(include=[np.number])
y = merged["averageRating"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    random_state=42
)

rf.fit(X_train, y_train)
preds = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, preds))
print("R^2:", r2_score(y_test, preds))


MSE: 1.813733409004537
R^2: 0.08272766606836168


In [ ]:
# crew_df = pd.read_csv("data/title.crew.tsv", sep='\t')
# print(crew_df.columns)


# print(crew_df)

In [ ]:
# For genres, make a seperate column for each individual one